# Functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import numpy as np
import pandas as pd
from math import pow, sqrt
from skimage.color import rgb2hsv 
from skimage import io 
from skimage.feature import local_binary_pattern
from google.colab import data_table
data_table.enable_dataframe_formatter()

def getEnergy(img):
    # Convert BGR to HSV and parse HSV
    hsv_img = rgb2hsv(img)
    h, s, v = hsv_img[:, :, 0], hsv_img[:, :, 1], hsv_img[:, :, 2]
    # Get height, width and number of pixels of the image
    height = img.shape[0]
    width = img.shape[1]
    pixels = (height-2) * (width-2)
    energy_h = [0] * pixels
    energy_s = [0] * pixels
    energy_v = [0] * pixels
    pixel = 0
        
    #Calculate the energy of each pixel (channel selected by user)
    #print('Getting energy from H channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = h[j+1][k]
            x2 = h[j+1][k+2]
            y1 = h[j][k+1]
            y2 = h[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_h[pixel] = energy
            pixel += 1
            
    pixel = 0               
    #print('Getting energy from S channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = s[j+1][k]
            x2 = s[j+1][k+2]
            y1 = s[j][k+1]
            y2 = s[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_s[pixel] = energy
            pixel += 1
            
    pixel = 0              
    #print('Getting energy from V channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = v[j+1][k]
            x2 = v[j+1][k+2]
            y1 = v[j][k+1]
            y2 = v[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_v[pixel] = energy
            pixel += 1
    
    # get histogram from eHSV
    (histH, _) = np.histogram(energy_h)
    (histS, _) = np.histogram(energy_s)
    (histV, _) = np.histogram(energy_v)
    histHSV = np.hstack((histH,histS,histV))

    return histHSV #Array of 1 x 30

def lbphist(img,n_points, radius, METHOD):
  n_points = n_points - 2
  lbp = local_binary_pattern(img, n_points, radius, METHOD)
  # get histogram from LBP
  (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, n_points + 3),range=(0, n_points + 2))
  # normalize the histogram
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)   
  #print(hist.shape)
  return hist

def extract_features(path_folder,details):
    #print(path_folder)
    list_images = os.listdir(path_folder)
    #print(list_images)

  # Settings for Local Binary Pattern
    radius = 3
    n_points = 10
    METHOD = 'default'
    cont = 0
    for image in list_images:
      #print(name_image[-3:])
      if(image[-4:] == '.png'):
        #print(image)
        path_image = os.path.join(path_folder, image)
        img = io.imread(path_image, as_gray=False)
        img_gray = io.imread(path_image, as_gray=True)
        energy_HSV = getEnergy(img)
        hist = lbphist(img_gray,n_points, radius, METHOD)
        #details = ['SUR_AU','test','SUR','AU', 1]
        FOLDER = details[0]
        SET = details[1]
        VIEW = details[2]
        CLASS = details[3]
        LABEL = details[4]
        
        features = np.hstack((image, FOLDER, SET, VIEW, CLASS, LABEL, energy_HSV, hist))
        #print(features)
        if cont == 0:
          list_features = features
          cont += 1
        else:
          list_features = np.vstack((list_features,features))
    
    #return list_features

    df_features = pd.DataFrame(list_features, columns = ['image', 'folder', 'set', 'view', 'class','label','h0','h1','h2','h3','h4','h5','h6','h7','h8','h9','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','lbp0','lbp1','lbp2','lbp3','lbp4','lbp5','lbp6','lbp7','lbp8','lbp9'])
    return df_features


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Extract features

In [3]:
#Test:

#dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'
#path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_AU/test'
#details = ['SUR_AU','test','SURFACE','AU',1]
#test_AU = extract_features(path_data,details)
#test_AU

# Surface

## Surface test

In [4]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_AU/test'
details = ['SUR_AU','test','SURFACE','AU',0]
test_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_BRU/test'
details = ['SUR_BRU','test','SURFACE','BRU',1]
test_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_CYS/test'
details = ['SUR_CYS','test','SURFACE','CYS',2]
test_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_STR/test'
details = ['SUR_STR','test','SURFACE','STR',3]
test_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_WD/test'
details = ['SUR_WD','test','SURFACE','WD',4]
test_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_WW/test'
details = ['SUR_WW','test','SURFACE','WW',5]
test_WW = extract_features(path_data,details)

SUR_test = pd.concat([test_AU,test_BRU,test_CYS,test_STR,test_WD,test_WW], ignore_index=True)
SUR_test

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,13216 - surface 1 - 122.png,SUR_AU,test,SURFACE,AU,0,64421,7,0,0,...,0.33585858585434525,0.08585858585750179,0.044191919191361215,0.10631313131178897,0.08282828282723702,0.00997474747462153,0.10568181818048382,0.1844696969673678,0.03446969696926175,0.010353535353404627
1,10237 - surface 3 - 2.png,SUR_AU,test,SURFACE,AU,0,62861,267,132,156,...,0.28452151573082013,0.11929993577200868,0.03853564547144291,0.08349389852145964,0.08108542067949445,0.03195247270340475,0.06278098908055907,0.2634874759109909,0.021034039819829254,0.01380860629393371
2,141888 - surface 1 - 350.png,SUR_AU,test,SURFACE,AU,0,63732,176,0,0,...,0.4233324107331396,0.07486853030618781,0.048574591751334426,0.08275671187264383,0.09576529199978183,0.00802657071674472,0.08995294768765634,0.1206753390517482,0.04691392194787,0.009133683919054336
3,13216 - surface 1 - 76.png,SUR_AU,test,SURFACE,AU,0,45269,14478,2935,1113,...,0.36775621825279653,0.08034900377548705,0.04414637322510553,0.1045709076689078,0.08646959239371703,0.008464643833722299,0.0901159005067051,0.17762729521841872,0.03125406953989772,0.009245995572219742
4,141888 - surface 1 - 386.png,SUR_AU,test,SURFACE,AU,0,63746,174,0,0,...,0.3580737795409976,0.07002429865175586,0.043406229290441724,0.09730505853658819,0.09410205433958357,0.010713496796877473,0.09752595537776092,0.17826375082639426,0.04141815771988714,0.009167218908668354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,17965 - surface 4 - 73.png,SUR_WW,test,SURFACE,WW,5,43056,15783,4212,1064,...,0.32462410647862183,0.0674143455747176,0.04633965984660661,0.11239832388325859,0.07468572836979682,0.0056692137046380426,0.11929997534977446,0.20335223070984285,0.038452058170588464,0.007764357899830363
1196,7806 - surface1 - 204.png,SUR_WW,test,SURFACE,WW,5,57006,6187,1072,137,...,0.3745449981125324,0.08673277268624661,0.04355466298426566,0.10330111710677418,0.07568720973922823,0.005648299234270763,0.10028869084849644,0.1604116982532897,0.040542236725987925,0.009288314296356365
1197,19882 - surface 3 - 143.png,SUR_WW,test,SURFACE,WW,5,49105,12957,1785,444,...,0.37187713310072523,0.08232081911150416,0.045733788395280084,0.09488054607379003,0.06894197952124312,0.0053242320818385775,0.10293515358221249,0.17474402730136868,0.04491467576730492,0.008327645051080851
1198,7806 - surface1 - 218.png,SUR_WW,test,SURFACE,WW,5,44561,14750,3730,892,...,0.3538042066949649,0.08750973772932343,0.05024668917099134,0.1108802908321101,0.06803427681033454,0.009348221241114668,0.09049597507023506,0.1778758763934319,0.039989613086990526,0.011815112957519929


## Surface train

In [5]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_AU/train'
details = ['SUR_AU','train','SURFACE','AU',0]
train_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_BRU/train'
details = ['SUR_BRU','train','SURFACE','BRU',1]
train_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_CYS/train'
details = ['SUR_CYS','train','SURFACE','CYS',2]
train_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_STR/train'
details = ['SUR_STR','train','SURFACE','STR',3]
train_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_WD/train'
details = ['SUR_WD','train','SURFACE','WD',4]
train_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SUR_WW/train'
details = ['SUR_WW','train','SURFACE','WW',5]
train_WW = extract_features(path_data,details)

SUR_train = pd.concat([train_AU,train_BRU,train_CYS,train_STR,train_WD,train_WW], ignore_index=True)
SUR_train

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,19718 - surface 2 - 794.png,SUR_AU,train,SURFACE,AU,0,38047,19873,5487,796,...,0.34244914454610736,0.06318200188517874,0.04890206116059677,0.1185504512984164,0.07099555435711982,0.006196955408780858,0.11181463020191547,0.1935874983134368,0.03825946382812529,0.006062238986850839
1,26599 - surface 2 - 903.png,SUR_AU,train,SURFACE,AU,0,40754,10938,8523,2584,...,0.41481637377233693,0.10091550036724281,0.038724613279609334,0.08849836893519417,0.09523308428816969,0.013258970851170587,0.07765968641399916,0.1257497632313401,0.03588340524007278,0.009260233610341362
2,19718 - surface 1 - 643.png,SUR_AU,train,SURFACE,AU,0,48017,13654,1946,561,...,0.3164248105776753,0.06472325670906995,0.04058045460330576,0.11390779504155762,0.0675484782320849,0.0034673173237001757,0.1094131244367611,0.24258379349887527,0.03621420315864628,0.005136766405481742
3,16927 - surface 1 - 592.png,SUR_AU,train,SURFACE,AU,0,61312,2848,257,61,...,0.2886864597358447,0.06488978857326934,0.036774628879478466,0.094916779126238,0.06792622582019875,0.005960413855083666,0.11729644624249554,0.27744039585832836,0.037336932073354284,0.008771929824462753
4,27340 - surface 1 - 1001.png,SUR_AU,train,SURFACE,AU,0,62877,237,46,141,...,0.22094541655496286,0.08957952467890337,0.038130060067404936,0.10851397231511473,0.08944894228141226,0.029250457038009266,0.09610864455345902,0.2969443718947905,0.02050143640610471,0.010577174196780136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,17965 - surface 2 - 122.png,SUR_WW,train,SURFACE,WW,5,50940,10784,1896,572,...,0.30269717499293497,0.07298990157135384,0.04397290042127096,0.11811325578271618,0.06429758404621888,0.004985299757062696,0.11606800460033148,0.22996292981937924,0.03911542886310731,0.007797520132841653
4796,9996 - surface 1 - 954.png,SUR_WW,train,SURFACE,WW,5,63293,221,13,8,...,0.3585873387989656,0.08147713950642618,0.042790152402655476,0.10052754982267692,0.09686400937724408,0.005422039859240592,0.0905627198110996,0.1821512309469204,0.034144196951433996,0.007473622508682978
4797,17965 - surface 2 - 85.png,SUR_WW,train,SURFACE,WW,5,64308,49,6,7,...,0.31725391498437844,0.07368568232559164,0.04432326621861964,0.10766219239223068,0.06487695749350005,0.004614093959667029,0.10794183445039232,0.23615771811750338,0.03411633109571985,0.009368008948414877
4798,24994 - surface 4 - 396.png,SUR_WW,train,SURFACE,WW,5,50934,10413,2472,515,...,0.30999668983267137,0.04634227077050079,0.03028798411072016,0.08374710360669071,0.09400860642015875,0.0043032108572607876,0.11999337967361812,0.270936775897535,0.036577292286716696,0.003806686527576851


# Section

## Section test

In [6]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_AU/test'
details = ['SEC_AU','test','SECTION','AU',0]
test_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_BRU/test'
details = ['SEC_BRU','test','SECTION','BRU',1]
test_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_CYS/test'
details = ['SEC_CYS','test','SECTION','CYS',2]
test_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_STR/test'
details = ['SEC_STR','test','SECTION','STR',3]
test_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_WD/test'
details = ['SEC_WD','test','SECTION','WD',4]
test_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_WW/test'
details = ['SEC_WW','test','SECTION','WW',5]
test_WW = extract_features(path_data,details)

SEC_test = pd.concat([test_AU,test_BRU,test_CYS,test_STR,test_WD,test_WW], ignore_index=True)
SEC_test

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,27385 - section 1 - 531.png,SEC_AU,test,SECTION,AU,0,42311,7648,8339,4668,...,0.35607166555599107,0.07086927670775224,0.05069674850629467,0.107232913070906,0.10006635699933555,0.005573988055665906,0.11068347710536584,0.15885865958647832,0.0349037823485746,0.005043132050364391
1,141888 - section 2 - 435.png,SEC_AU,test,SECTION,AU,0,64496,3,4,3,...,0.37414201921410034,0.08635966554241409,0.04717334331652099,0.10058654686009502,0.07013602895207617,0.00611506302251198,0.1057032322462785,0.15325096717642267,0.04679895170289781,0.009734181954202743
2,27385 - section 1 - 586.png,SEC_AU,test,SECTION,AU,0,61937,2207,157,109,...,0.34194986072042394,0.06997214484601702,0.046016713091409284,0.09972144846685546,0.08512534818846657,0.0064623955431034824,0.09905292478998269,0.20022284122339584,0.04278551531985754,0.008690807799346063
3,141888 - section 2 - 459.png,SEC_AU,test,SECTION,AU,0,64283,45,4,18,...,0.37397938695791755,0.08258599919578924,0.0460447061966799,0.10440369428316955,0.06772855039395358,0.005889439164691615,0.09717574621741165,0.16584125284211162,0.04765091687795943,0.008700307856930795
4,139043 - section 5 - 289.png,SEC_AU,test,SECTION,AU,0,53394,8334,1781,661,...,0.3415323055155757,0.07696834176677687,0.04954689890491294,0.10097681534540452,0.07967517947417117,0.007885135930235551,0.10450747322461447,0.1960692008921258,0.03436507002431016,0.008473578910103877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,14811 - section 2 - 63.png,SEC_WW,test,SECTION,WW,5,62774,477,68,109,...,0.32115030457116944,0.07593143504638147,0.035415781271633154,0.0968975775591883,0.06672333191575686,0.0056665250034613045,0.0990225244354863,0.24026066014675931,0.050007083155546014,0.008924776880451554
1196,8234 - section 1 - 154.png,SEC_WW,test,SECTION,WW,5,64144,72,6,8,...,0.3555455500471928,0.08824853669385789,0.03962179198499742,0.10430736905141368,0.07504127269885875,0.004652558907329243,0.09275101305578942,0.18910400720112405,0.04262344289295177,0.008104457451476746
1197,4966 - section 2 - 149.png,SEC_WW,test,SECTION,WW,5,64157,10,0,0,...,0.34445277360888993,0.07096451774024283,0.03523238380765577,0.09757621189283389,0.07546226886462441,0.007371314342736491,0.09207896051858971,0.2211394302820947,0.0468515742123082,0.008870564717530354
1198,14811 - section 2 - 47.png,SEC_WW,test,SECTION,WW,5,61166,1677,257,381,...,0.3248048568903613,0.05767562879361556,0.039462272332473805,0.08933217692845719,0.08860942468793569,0.005492917027963387,0.11274934952135372,0.23084706562256654,0.04336513443128989,0.007661173749527881


## Section train

In [7]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_AU/train'
details = ['SEC_AU','train','SECTION','AU',0]
train_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_BRU/train'
details = ['SEC_BRU','train','SECTION','BRU',1]
train_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_CYS/train'
details = ['SEC_CYS','train','SECTION','CYS',2]
train_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_STR/train'
details = ['SEC_STR','train','SECTION','STR',3]
train_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_WD/train'
details = ['SEC_WD','train','SECTION','WD',4]
train_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/SEC_WW/train'
details = ['SEC_WW','train','SECTION','WW',5]
train_WW = extract_features(path_data,details)

SEC_train = pd.concat([train_AU,train_BRU,train_CYS,train_STR,train_WD,train_WW], ignore_index=True)
SEC_train

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,13216 - section 1 - 231.png,SEC_AU,train,SECTION,AU,0,55357,5801,1889,890,...,0.3803342079770704,0.07899461400249973,0.03590664272840897,0.08990470929305476,0.06987985084836515,0.007871840905843504,0.08659024996427855,0.19776273995031401,0.04198315149783202,0.01077199281852269
1,13216 - section 1 - 178.png,SEC_AU,train,SECTION,AU,0,33471,9775,12004,6133,...,0.3898727862261178,0.09478672985663773,0.03778997256126478,0.09565976552636993,0.06747318533215922,0.0053629333997834506,0.08992267398241553,0.15265652282174288,0.05350461461644419,0.012970815664592532
2,13216 - section 1 - 238.png,SEC_AU,train,SECTION,AU,0,56608,5029,1613,734,...,0.3700961288598325,0.0756040529997973,0.0366328916596956,0.09327097947397674,0.0694985710565147,0.0061054819432826,0.09599896076778386,0.19940244219018705,0.04247856586071085,0.010911925175228477
3,13216 - section 1 - 195.png,SEC_AU,train,SECTION,AU,0,35492,9455,11922,5204,...,0.36297109403143096,0.07781436760485651,0.04890840346324053,0.10550067081222708,0.07391145261527123,0.007439931698896939,0.1064763995596234,0.16575192096394983,0.042932064885438076,0.00829369435286872
4,16927 - section 2 - 711.png,SEC_AU,train,SECTION,AU,0,33963,12939,9974,5186,...,0.35319820366120647,0.07889306954631758,0.043087753367604224,0.10401747784799104,0.0731884937483531,0.005825949751112684,0.09297244811150658,0.19431969899023763,0.04466561475853058,0.009831290205002653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,20135 - section1 - 671.png,SEC_WW,train,SECTION,WW,5,53992,7686,1962,560,...,0.37612883804370667,0.07179409993871472,0.0385310054178427,0.09030704394806884,0.07962071041421402,0.005719446116711026,0.1037025888003657,0.17986152919657042,0.04695966285299579,0.007375075255758955
4796,14811 - section 1 - 260.png,SEC_WW,train,SECTION,WW,5,45427,13704,3406,1311,...,0.32747089422433157,0.0823631409452271,0.04012880852068208,0.10502848649857532,0.06006935843373706,0.005945008669730679,0.10131285607999364,0.22516720336604945,0.0426058954664032,0.009908347782884465
4797,14811 - section 1 - 135.png,SEC_WW,train,SECTION,WW,5,49538,10541,2501,1099,...,0.3177496037992902,0.06973058636991904,0.035129424193936484,0.0965398837810811,0.06444796619037972,0.006471209719935668,0.10433174854590159,0.25765979925703036,0.036978341256775245,0.01096143687254409
4798,14811 - section 1 - 283.png,SEC_WW,train,SECTION,WW,5,64300,57,3,3,...,0.31900741877310046,0.0892811460720225,0.04400102327904834,0.10680480941280629,0.060117677154513716,0.00537221795848846,0.10322333077381397,0.22819135328436696,0.03504732668156757,0.008953696597480767


# Mixed

## Mixed test

In [8]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_AU/test'
details = ['MIX_AU','test','MIXED','AU',0]
test_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_BRU/test'
details = ['MIX_BRU','test','MIXED','BRU',1]
test_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_CYS/test'
details = ['MIX_CYS','test','MIXED','CYS',2]
test_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_STR/test'
details = ['MIX_STR','test','MIXED','STR',3]
test_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_WD/test'
details = ['MIX_WD','test','MIXED','WD',4]
test_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_WW/test'
details = ['MIX_WW','test','MIXED','WW',5]
test_WW = extract_features(path_data,details)

MIX_test = pd.concat([test_AU,test_BRU,test_CYS,test_STR,test_WD,test_WW], ignore_index=True)
MIX_test

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,141888 - section 1 - 164.png,MIX_AU,test,MIXED,AU,0,37234,10732,10503,4418,...,0.40919216789296287,0.08265916285516872,0.04725055692288368,0.09438386680625649,0.0780865283142445,0.007621057568207045,0.08934224410728875,0.14362762340082508,0.03798804080152435,0.00984875131891372
1,139043 - section 4 - 67.png,MIX_AU,test,MIXED,AU,0,52426,10163,1284,425,...,0.3266754270653525,0.07963206307385504,0.045860709592038626,0.11287779237696613,0.06872536136571977,0.00670170827849275,0.1030223390262415,0.20643889618651198,0.042575558475130414,0.007490144546550721
2,139043 - section 4 - 16.png,MIX_AU,test,MIXED,AU,0,53042,8989,1589,495,...,0.34539989544765554,0.07710402509047172,0.047830632513750256,0.1071615263969268,0.07566649241929344,0.006534239414446756,0.10376372190141449,0.1945896497622244,0.036461055932612896,0.005488761108135275
3,139043 - section 4 - 84.png,MIX_AU,test,MIXED,AU,0,54160,8915,1021,235,...,0.3125634517726832,0.08058375634415503,0.0456852791872375,0.11205583756202975,0.06281725888245156,0.006852791878085625,0.09327411167394323,0.2326142131950176,0.04403553299436503,0.009517766497341146
4,139043 - section 4 - 46.png,MIX_AU,test,MIXED,AU,0,48986,12093,2180,800,...,0.3514150943347473,0.07672031076475139,0.043146503883974105,0.09947280798974094,0.08171476137511495,0.004994450610363562,0.10335738068669038,0.1884017758020477,0.04550499444997912,0.005271920088717093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,26319 - surface 2 - 77.png,MIX_WW,test,MIXED,WW,5,64498,0,0,0,...,0.333859510650699,0.06682451986231486,0.039331754800850674,0.10247303341091196,0.0789265982625766,0.00736648250450715,0.10655090765447842,0.21678505656121042,0.03972638779216356,0.008155748487132916
1196,26319 - surface 2 - 86.png,MIX_WW,test,MIXED,WW,5,42850,12143,5349,2380,...,0.32094111529545116,0.07799298063072933,0.03730664240169886,0.1062004419588431,0.07253347198657828,0.008059274665175364,0.1012608865188969,0.22968932795749786,0.036396724294340356,0.00961913427778995
1197,26319 - surface 2 - 98.png,MIX_WW,test,MIXED,WW,5,64502,0,0,0,...,0.31722383720545994,0.07945736434012285,0.032703488371696905,0.09435562015389588,0.0726744186037709,0.0095687984494965,0.09484011627792102,0.25121124030703473,0.03645833333289173,0.011506782945597059
1198,25962 - section 1 - 24.png,MIX_WW,test,MIXED,WW,5,64405,7,0,0,...,0.33694594969658265,0.09101014311392233,0.040016673613449814,0.1104626927871271,0.061970265387495206,0.004724190634921159,0.10490482145192574,0.20077810198414925,0.042378768930910395,0.006808392385621671


## Mixed train

In [9]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/'

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_AU/train'
details = ['MIX_AU','train','MIXED','AU',0]
train_AU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_BRU/train'
details = ['MIX_BRU','train','MIXED','BRU',1]
train_BRU = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_CYS/train'
details = ['MIX_CYS','train','MIXED','CYS',2]
train_CYS = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_STR/train'
details = ['MIX_STR','train','MIXED','STR',3]
train_STR = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_WD/train'
details = ['MIX_WD','train','MIXED','WD',4]
train_WD = extract_features(path_data,details)

path_data = dir_root + 'Yonathan El Beze (patches - 6 classes - w256 - 1k per class and view)/MIX_WW/train'
details = ['MIX_WW','train','MIXED','WW',5]
train_WW = extract_features(path_data,details)

MIX_train = pd.concat([train_AU,train_BRU,train_CYS,train_STR,train_WD,train_WW], ignore_index=True)
MIX_train

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,16927 - surface 2 - 689.png,MIX_AU,train,MIXED,AU,0,62552,1373,323,140,...,0.31635474265933267,0.06741811994824097,0.03860059549086983,0.10814547001161053,0.07890259463973943,0.008932369204498804,0.0888983411304881,0.2421310080790926,0.04179072734961941,0.00882603147587382
1,13216 - surface 2 - 254.png,MIX_AU,train,MIXED,AU,0,55731,6565,1364,548,...,0.34097669806220904,0.0796975706484916,0.04586018839804338,0.11812097173874417,0.05528011898791175,0.0053296975705834195,0.09692612791154033,0.20637084779119522,0.04053049082745996,0.010907288051426534
2,16927 - section 1 - 457.png,MIX_AU,train,MIXED,AU,0,62088,1619,485,174,...,0.33111054116015504,0.059502436521422127,0.032828930494577725,0.08412413439235543,0.07925109002206654,0.006283662477477768,0.1023339317760665,0.2545524493427218,0.04308797127413327,0.006924852526199989
3,6109 - surface 4 - 865.png,MIX_AU,train,MIXED,AU,0,59940,943,453,218,...,0.278608618585592,0.09471585244162273,0.033565968760013665,0.06934751301573781,0.08751523208056369,0.030131826741662433,0.07433255788108638,0.2905727262624285,0.027694693696380914,0.013515010523833885
4,28385 - surface 1 - 812.png,MIX_AU,train,MIXED,AU,0,50733,2439,7168,2591,...,0.36456456456091524,0.08218218218135953,0.04054054054013473,0.10280280280177374,0.0789789789781884,0.010510510510405299,0.0891891891882964,0.1886886886867999,0.03463463463428794,0.007907907907828748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,25962 - surface 2 - 465.png,MIX_WW,train,MIXED,WW,5,51932,9045,2454,693,...,0.3636645321177064,0.07880603181079522,0.043689320387898266,0.083453831852061,0.08469324519639854,0.00826275562891693,0.10948151208314932,0.1784755215846057,0.03862838256518665,0.010844866762953471
4796,25962 - surface 1 - 159.png,MIX_WW,train,MIXED,WW,5,48425,9630,4183,1477,...,0.4147602429828776,0.09473956313694275,0.04678816078522957,0.10120201628407391,0.07819568308028699,0.006850200335959025,0.08207315496856568,0.1346775235862133,0.03360475636508201,0.007108698461844272
4797,24994 - surface 3 - 96.png,MIX_WW,train,MIXED,WW,5,63406,243,10,16,...,0.32936352986471373,0.07828973057246603,0.046661460366133126,0.08238969152513882,0.09839906286414683,0.013080827801384599,0.07926591175167384,0.23447871924571498,0.030456852791283545,0.007614213197820886
4798,27718 - surface 4 - 802.png,MIX_WW,train,MIXED,WW,5,60578,586,209,32,...,0.3409763313559027,0.07085798816463228,0.05177514792822818,0.10591715976174679,0.07869822485090683,0.0035502958579356468,0.11582840236515048,0.18550295857713753,0.04068047337217928,0.0062130177513873815


# Save

In [10]:
# For UMAPS

SEC = pd.concat([SEC_train, SEC_test], ignore_index=True)
SUR = pd.concat([SUR_train, SUR_test], ignore_index=True)
MIX = pd.concat([MIX_train, MIX_test], ignore_index=True)

with pd.ExcelWriter('features_JElBeze_UMAPS.xlsx') as writer:
    SEC.to_excel(writer, sheet_name='section',index = False)
    SUR.to_excel(writer, sheet_name='surface',index = False)
    MIX.to_excel(writer, sheet_name='mixed',index = False)

In [11]:
# For XGBOOST

with pd.ExcelWriter('features_JElBeze_train.xlsx') as writer:
    SEC_train.to_excel(writer, sheet_name='section',index = False)
    SUR_train.to_excel(writer, sheet_name='surface',index = False)
    MIX_train.to_excel(writer, sheet_name='mixed',index = False)

with pd.ExcelWriter('features_JElBeze_test.xlsx') as writer:
    SEC_test.to_excel(writer, sheet_name='section',index = False)
    SUR_test.to_excel(writer, sheet_name='surface',index = False)
    MIX_test.to_excel(writer, sheet_name='mixed',index = False)